In [1]:
import pickle
with open("extracted_result.pkl", "rb") as file:
    dict = pickle.load(file)


In [2]:
print(dict["relationships"])
nodes = dict["nodes"]
relationships = dict["relationships"]
print(len(nodes))


[{'start': 'quyền_và_nghĩa_vụ_người_sử_dụng_đất', 'end': 'VănBảnPhápLuật', 'type': 'liênQuanĐến', 'properties': {}}, {'start': 'nhà_nước', 'end': 'quyền_và_nghĩa_vụ_người_sử_dụng_đất', 'type': 'thựcHiệnQuyềnĐạiDiệnChủSởHữuVềĐấtĐai', 'properties': {}}, {'start': 'quốc_hội', 'end': 'quyền_và_nghĩa_vụ_người_sử_dụng_đất', 'type': 'thựcHiệnQuyềnĐạiDiệnChủSởHữuVềĐấtĐai', 'properties': {}}, {'start': 'ủy_ban_thường_vụ_quốc_hội', 'end': 'quyền_và_nghĩa_vụ_người_sử_dụng_đất', 'type': 'thựcHiệnQuyềnĐạiDiệnChủSởHữuVềĐấtĐai', 'properties': {}}, {'start': 'phòng_hộ', 'end': 'đất_rừng_sản_xuất', 'type': 'liênQuanĐến', 'properties': {}}, {'start': 'luật', 'end': 'đất_rừng_sản_xuất', 'type': 'liênQuanĐến', 'properties': {}}, {'start': 'luật', 'end': 'giám_sát_việc_thi_hành_pháp_luật_về_đất_đai', 'type': 'liênQuanĐến', 'properties': {}}, {'start': 'chính_phủ', 'end': 'nhà_nước', 'type': 'thựcHiệnQuyềnĐạiDiệnChủSởHữu', 'properties': {'thẩm_quyền': 'theo quy định của Luật'}}, {'start': 'ủy_ban_nhân_dân',

In [3]:
label = set()
for node in dict["nodes"]:
    label.add(node["label"])
print(list(label))

['ĐấtĐai', 'HoạtĐộngLiênQuanĐếnĐấtĐai', 'QuyềnVàNghĩaVụ', 'ChủSởHữuĐấtĐai', 'CơQuanNhàNước', 'Vùng', 'VănBảnPhápLuật', 'NgườiSửDụngĐất']


In [4]:
def correct_key(key):
    if '_' in key:
        return key
    else:
        return '_'.join(key.split())

def correct_value(value):
    if ' ' in value:
        return ''.join(word.capitalize() for word in value.split())
    else:
        return value

# Correct keys in nodes list
for node in nodes:
    node['name'] = correct_key(node['name'])
    node['label'] = correct_value(node['label'])
    node['properties'] = {correct_key(k): v for k, v in node['properties'].items()}

# Correct keys in relationships list
for rel in relationships:
    rel['start'] = correct_key(rel['start'])
    rel['end'] = correct_key(rel['end'])
    rel['type'] = correct_value(rel['type'])
    rel['properties'] = {correct_key(k): v for k, v in rel['properties'].items()}

In [5]:
# Write nodes to CSV file
with open('nodes.csv', 'w', encoding='utf-8') as file:
    # Get all unique property keys from the nodes
    property_keys = set()
    for node in nodes:
        property_keys.update(node['properties'].keys())

    # Write the header row with property keys
    header = ':ID,name,:LABEL,' + ','.join(property_keys)
    file.write(header + '\n')

    # Write node rows with properties
    for node in nodes:
        node_id = node['name']
        name = node['name']
        label = node['label']
        properties = node['properties']

        # Write property values or empty strings if not present
        property_values = [str(properties.get(key, '')) for key in property_keys]
        row = f"{node_id},{name},{label}," + ','.join(property_values)
        file.write(row + '\n')

# Write relationships to CSV file
with open('relationships.csv', 'w', encoding='utf-8') as file:
    # Get all unique property keys from the relationships
    property_keys = set()
    for rel in relationships:
        property_keys.update(rel['properties'].keys())

    # Write the header row with property keys
    header = ':START_ID,:END_ID,:TYPE,' + ','.join(property_keys)
    file.write(header + '\n')

    # Write relationship rows with properties
    for rel in relationships:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        properties = rel['properties']

        # Write property values or empty strings if not present
        property_values = [str(properties.get(key, '')) for key in property_keys]
        row = f"{start},{end},{rel_type}," + ','.join(property_values)
        file.write(row + '\n')

print("CSV files generated: nodes.csv and relationships.csv")

CSV files generated: nodes.csv and relationships.csv


In [6]:
node_cyphers = []
for node in nodes:
    name = node['name']
    label = node['label']
    properties = node['properties']

    cypher = f"CREATE (:{label} {{name: '{name}'"
    for key, value in properties.items():
        if value is not None:
            cypher += f", {key}: '{value}'"
    cypher += "})"

    node_cyphers.append(cypher)

In [7]:
rel_cyphers = []
for rel in dict["relationships"]:
    start = rel['start']
    end = rel['end']
    rel_type = rel['type']
    properties = rel['properties']

    cypher = f"MATCH (a {{name: '{start}'}}) \nMATCH(b {{name: '{end}'}}) \nCREATE (a)-[:{rel_type}"
    if properties:
        cypher += " {"
        for key, value in properties.items():
            cypher += f"{key}: '{value}', "
        cypher = cypher[:-2]  # Remove the trailing comma and space
        cypher += "}"
    cypher += "]->(b)"

    rel_cyphers.append(cypher)

In [8]:
print(set(rel_cyphers))

{"MATCH (a {name: 'chính_quyền_địa_phương'}) \nMATCH(b {name: 'đất_đai'}) \nCREATE (a)-[:cóTráchNhiệmQuảnLý]->(b)", "MATCH (a {name: 'hệ_thống_cơ_quan_quản_lý_đất_đai'}) \nMATCH(b {name: 'trung_ương'}) \nCREATE (a)-[:tổChứcTại]->(b)", "MATCH (a {name: 'mua'}) \nMATCH(b {name: 'ĐấtĐai'}) \nCREATE (a)-[:liên_quan_đến]->(b)", "MATCH (a {name: 'Luật'}) \nMATCH(b {name: 'QuyĐịnhKhác'}) \nCREATE (a)-[:liênQuan]->(b)", "MATCH (a {name: 'người_sử_dụng_đất'}) \nMATCH(b {name: 'thu_hồi_đất'}) \nCREATE (a)-[:thamGiaÝKiếnVề]->(b)", "MATCH (a {name: 'Chính_phủ'}) \nMATCH(b {name: 'ĐấtĐai'}) \nCREATE (a)-[:thốngNhấtQuảnLý {phạm_vi: 'cả_nước'}]->(b)", "MATCH (a {name: 'người_sử_dụng_đất'}) \nMATCH(b {name: 'bồi_thường_hỗ_trợ_tái_định_cư'}) \nCREATE (a)-[:thựcHiện {quyền: 'thực hiện bồi thường, hỗ trợ, tái định cư'}]->(b)", "MATCH (a {name: 'quỹ_đất'}) \nMATCH(b {name: 'nhà_nước'}) \nCREATE (a)-[:thuộc]->(b)", "MATCH (a {name: 'văn_bản_pháp_luật'}) \nMATCH(b {name: 'quyền_và_nghĩa_vụ'}) \nCREATE (a)-[

In [9]:
from neo4j import GraphDatabase
uri = "neo4j+s://94b452c3.databases.neo4j.io"
user = "neo4j"
password = "r3vGHlBAZzm3dymZLBlLBk-RNFzR5xRlGIb-tSNQwvY"
driver = GraphDatabase.driver(uri, auth=(user, password))

In [10]:
def execute_cypher_queries(tx, cypher_queries):
    for query in cypher_queries:
        tx.run(query)
with driver.session() as session:
    session.execute_write(execute_cypher_queries, list(set(node_cyphers)))
    session.execute_write(execute_cypher_queries, list(set(rel_cyphers)))